In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict, Image
import PIL

In [2]:
def prep_dataset(sample):
    img = sample['image']
    sample['width'] = img.width
    sample['height'] = img.height

    if (img.mode == 'L') | (img.mode == 'CMYK') | (img.mode == 'RGBA'):
        rgbimg = PIL.Image.new("RGB", img.size)
        rgbimg.paste(img)
        img = rgbimg
    
    sample['image'] = Image().encode_example(img)

    return sample

In [3]:
batch_size = 64

# Training split

Cutoff for training split is 2048 samples

In [5]:
%%time
dataset = load_dataset("imagenet-1k",split='train')
dataset = dataset.map(prep_dataset)

CPU times: user 1.22 s, sys: 2.09 s, total: 3.31 s
Wall time: 1.47 s


In [6]:
train_dataset = Dataset.from_dict({
    "img_batch" : [],
    "label_batch" : [],
    "width" : [],
    "height": [],
})

In [7]:
width = torch.tensor(dataset['width']);
height = torch.tensor(dataset['height']);
unique_pairs = torch.unique(torch.stack([width, height], dim=1), dim=0)
pair_counts = {(w.item(), h.item()): ((width == w) & (height == h)).sum().item() 
               for w, h in unique_pairs}
sizes = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)
N = 0;
while (sizes[N][1]>=2048):
    N +=1
sizes = sizes[:N]

In [8]:
%%time
for size, count in sizes:
    w = size[0]; h = size[1]
    filtered = dataset.filter(lambda x: x['width']==w and x['height']==h)
    for i_batch in range(len(filtered)//batch_size):
        ind = range(i_batch * batch_size, (i_batch + 1) * batch_size)
        img_batch = filtered[ind]['image']
        img_batch = [Image().encode_example(pil_img) for pil_img in img_batch] 
        label_batch = filtered[ind]['label']

        train_dataset = train_dataset.add_item({
            "img_batch" : img_batch,
            "label_batch" : label_batch,
            "width" : w,
            "height": h,
        })

CPU times: user 29min 37s, sys: 1min 1s, total: 30min 38s
Wall time: 32min 24s


# Test split

Cutoff for the test split is 256 samples

In [9]:
%%time
dataset = load_dataset("imagenet-1k",split='test')
dataset = dataset.map(prep_dataset)

CPU times: user 415 ms, sys: 75.6 ms, total: 491 ms
Wall time: 1.54 s


In [10]:
test_dataset = Dataset.from_dict({
    "img_batch" : [],
    "label_batch" : [],
    "width" : [],
    "height": [],
})

In [11]:
width = torch.tensor(dataset['width']);
height = torch.tensor(dataset['height']);
unique_pairs = torch.unique(torch.stack([width, height], dim=1), dim=0)
pair_counts = {(w.item(), h.item()): ((width == w) & (height == h)).sum().item() 
               for w, h in unique_pairs}
sizes = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)
N = 0;
while (sizes[N][1]>=256):
    N +=1
sizes = sizes[:N]

In [12]:
%%time
for size, count in sizes:
    w = size[0]; h = size[1]
    filtered = dataset.filter(lambda x: x['width']==w and x['height']==h)
    for i_batch in range(len(filtered)//batch_size):
        ind = range(i_batch * batch_size, (i_batch + 1) * batch_size)
        img_batch = filtered[ind]['image']
        img_batch = [Image().encode_example(pil_img) for pil_img in img_batch] 
        label_batch = filtered[ind]['label']

        test_dataset = test_dataset.add_item({
            "img_batch" : img_batch,
            "label_batch" : label_batch,
            "width" : w,
            "height": h,
        })

CPU times: user 1min 47s, sys: 10.3 s, total: 1min 57s
Wall time: 2min 3s


# Validation split

Cutoff for validation split is 128 samples

In [13]:
%%time
dataset = load_dataset("imagenet-1k",split='validation')
dataset = dataset.map(prep_dataset)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

CPU times: user 1min 37s, sys: 3.03 s, total: 1min 40s
Wall time: 1min 43s


In [14]:
val_dataset = Dataset.from_dict({
    "img_batch" : [],
    "label_batch" : [],
    "width" : [],
    "height": [],
})

In [ ]:
width = torch.tensor(dataset['width']);
height = torch.tensor(dataset['height']);
unique_pairs = torch.unique(torch.stack([width, height], dim=1), dim=0)
pair_counts = {(w.item(), h.item()): ((width == w) & (height == h)).sum().item() 
               for w, h in unique_pairs}
sizes = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)
N = 0;
while (sizes[N][1]>=128):
    N +=1
sizes = sizes[:N]

In [15]:
%%time
for size, count in sizes:
    w = size[0]; h = size[1]
    filtered = dataset.filter(lambda x: x['width']==w and x['height']==h)
    for i_batch in range(len(filtered)//batch_size):
        ind = range(i_batch * batch_size, (i_batch + 1) * batch_size)
        img_batch = filtered[ind]['image']
        img_batch = [Image().encode_example(pil_img) for pil_img in img_batch] 
        label_batch = filtered[ind]['label']

        val_dataset = val_dataset.add_item({
            "img_batch" : img_batch,
            "label_batch" : label_batch,
            "width" : w,
            "height": h,
        })

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

CPU times: user 9min 44s, sys: 2min 32s, total: 12min 17s
Wall time: 12min 16s


In [16]:
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": val_dataset,
})

In [17]:
dataset.push_to_hub("danjacobellis/imagenet_batched_64")

Pushing dataset shards to the dataset hub:   0%|          | 0/59 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]